# Event analysis

# Import libraries and files

In [1]:
#import libraries and files
import pandas as pd
import operator
import cv2
import csv
import math
import matplotlib.pyplot as plt  # standard Python plotting library
import numpy as np  # package for scientific computing, handles arrays and math
import matplotlib.gridspec as gs
from scipy.optimize import curve_fit
import seaborn as sns
from tdt import read_block

#photometry path and file
FOLDER_PATH = '/home/li/Documents/Template_matching/Photometry/'

#Using photometry sessions from 9 different mice
FOLDER_NAME = []
FOLDER_NAME.append('159459-R-221006-091824')
FOLDER_NAME.append('159459-L-221020-102241')
FOLDER_NAME.append('159459-BL-221018-091752')
FOLDER_NAME.append('151653-R-221005-112408')
FOLDER_NAME.append('151653-L-221019-114353')
FOLDER_NAME.append('151653-RR-221012-102704')
FOLDER_NAME.append('151652-L-221011-091017')
FOLDER_NAME.append('151652-RR-221018-121215')
FOLDER_NAME.append('151653-LL-221101-100649')
tpath=len(FOLDER_NAME)

print("FOLDER_NAME: ", FOLDER_NAME)
BLOCK_PATH = []
for i in range(tpath):
    BLOCK_PATH.append(FOLDER_PATH + FOLDER_NAME[i])
#end for i

#Path to videos
VIDEO_PATH = []
VIDEO_PATH.append(BLOCK_PATH[0]+'/'+'Li_photo_stim_1_camera_-220601-092827_159459-R-221006-091824_Cam1.avi')
VIDEO_PATH.append(BLOCK_PATH[1]+'/'+'Li_photo_stim_1_camera_-220601-092827_159459-L-221020-102241_Cam1.avi')
VIDEO_PATH.append(BLOCK_PATH[2]+'/'+'Li_photo_stim_1_camera_-220601-092827_159459-BL-221018-091752_Cam1.avi')
VIDEO_PATH.append(BLOCK_PATH[3]+'/'+'Li_photo_stim_1_camera_-220601-092827_151653-R-221005-112408_Cam1.avi')
VIDEO_PATH.append(BLOCK_PATH[4]+'/'+'Li_photo_stim_1_camera_-220601-092827_151653-L-221019-114353_Cam1.avi')
VIDEO_PATH.append(BLOCK_PATH[5]+'/'+'Li_photo_stim_1_camera_-220601-092827_151653-RR-221012-102704_Cam1.avi')
VIDEO_PATH.append(BLOCK_PATH[6]+'/'+'Li_photo_stim_1_camera_-220601-092827_151652-L-221011-091017_Cam1.avi')
VIDEO_PATH.append(BLOCK_PATH[7]+'/'+'Li_photo_stim_1_camera_-220601-092827_151652-RR-221018-121215_Cam1.avi')
VIDEO_PATH.append(BLOCK_PATH[8]+'/'+'Li_photo_stim_1_camera_-220601-092827_151653-LL-221101-100649_Cam1.avi')

#Behavior Deeplabcut file
BEHAVE_PATH = '/home/li/Documents/Template_matching/DLC/'
BEHAVE_NAME = []
BEHAVE_NAME.append('Li_photo_stim_1_camera_-220601-092827_159459-R-221006-091824_Cam1DLC_resnet50_Anesthesia_simplified_2Apr4shuffle1_300000.csv')
BEHAVE_NAME.append('Li_photo_stim_1_camera_-220601-092827_159459-L-221020-102241_Cam1DLC_resnet50_Anesthesia_simplified_2Apr4shuffle1_300000.csv')
BEHAVE_NAME.append('Li_photo_stim_1_camera_-220601-092827_159459-BL-221018-091752_Cam1DLC_resnet50_Anesthesia_simplified_2Apr4shuffle1_300000.csv')
BEHAVE_NAME.append('Li_photo_stim_1_camera_-220601-092827_151653-R-221005-112408_Cam1DLC_resnet50_Anesthesia_simplified_2Apr4shuffle1_300000.csv')
BEHAVE_NAME.append('Li_photo_stim_1_camera_-220601-092827_151653-L-221019-114353_Cam1DLC_resnet50_Anesthesia_simplified_2Apr4shuffle1_300000.csv')
BEHAVE_NAME.append('Li_photo_stim_1_camera_-220601-092827_151653-RR-221012-102704_Cam1DLC_resnet50_Anesthesia_simplified_2Apr4shuffle1_300000.csv')
BEHAVE_NAME.append('Li_photo_stim_1_camera_-220601-092827_151652-L-221011-091017_Cam1DLC_resnet50_Anesthesia_simplified_2Apr4shuffle1_300000.csv')
BEHAVE_NAME.append('Li_photo_stim_1_camera_-220601-092827_151652-RR-221018-121215_Cam1DLC_resnet50_Anesthesia_simplified_2Apr4shuffle1_300000.csv')
BEHAVE_NAME.append('Li_photo_stim_1_camera_-220601-092827_151653-LL-221101-100649_Cam1DLC_resnet50_Anesthesia_simplified_2Apr4shuffle1_300000.csv')

print("\033[1m", tpath, " Experiments: "+"\033[0m")
for i in range(tpath):
    print(i+1,".", FOLDER_NAME[i])
print("Done")


FOLDER_NAME:  ['159459-R-221006-091824', '159459-L-221020-102241', '159459-BL-221018-091752', '151653-R-221005-112408', '151653-L-221019-114353', '151653-RR-221012-102704', '151652-L-221011-091017', '151652-RR-221018-121215', '151653-LL-221101-100649']
 9  Experiments: 
1 . 159459-R-221006-091824
2 . 159459-L-221020-102241
3 . 159459-BL-221018-091752
4 . 151653-R-221005-112408
5 . 151653-L-221019-114353
6 . 151653-RR-221012-102704
7 . 151652-L-221011-091017
8 . 151652-RR-221018-121215
9 . 151653-LL-221101-100649
Done


In [18]:
#If prior saved arrays exist, can import them here

PATH = '/home/li/Documents/Template_matching/Archived/'
event_times=np.load(PATH+"20241220_event_init.npy")
event2_times=np.load(PATH+"20241220_event_arrest.npy")
behave_time = np.load(PATH+"20241220_behave_time.npy")
marker_dist=np.load(PATH+"20241220_dist_markers.npy")
movement_bin = np.load(PATH+"20241220_movement_bin.npy")
all_movement_bin = np.load(PATH+"20241220_all_movement_bin.npy")


In [3]:
PATH = '/home/li/Documents/Template_matching/Archived/'
all_movement_bin = np.load(PATH+"20241220_all_movement_bin.npy")

# TPR and TFR for movement onset predictions

In [6]:
#Finding true positive rate and true negative rate for the predictions
#select mouse
k=1
diff = 0.5 #the cutoff time difference in s below which is considered the same event

event3_times = np.load("20241220_init_peak_times.npy")
#Get video information
SELECT_VIDEO_PATH = VIDEO_PATH[k]
print("Collecting video information...")
cap = cv2.VideoCapture(SELECT_VIDEO_PATH)
fwidth = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
fheight = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
frate = cap.get(cv2.CAP_PROP_FPS)
tframes = cap.get(cv2.CAP_PROP_FRAME_COUNT)
cap.release()
cv2.destroyAllWindows()

#find all movement events
bin_pos = 0
bin_events = []
for i in range(len(all_movement_bin)):
    if(all_movement_bin[i]>0):
        if(bin_pos==0):
            bin_pos=1
            bin_events.append(i/10)
    else:
        bin_pos=0
    #endif
#endfor i
print("True movement events:", bin_events)
print("Predicted movement onset events:", np.round(event3_times,1))

#find nearest neighbor between peak events and movement events
count=0
for i in range(len(event3_times)):
    for j in range(len(bin_events)):
        if(abs(event3_times[i]-bin_events[j])<diff):
            count=count+1
            break
        #endif
    #endfor j
#endfor i
print("Total real events: ", len(bin_events), "Total predicted events: ", len(event3_times), "Count:", count) 
print("Sensitivity or TPR: ", np.round(count/len(bin_events),3))
count2=len(event3_times)-count
print("False positive rate: ", np.round(count2/(count2+300*frate-len(bin_events)),3))


Binary events: [1.9, 5.0, 6.5, 8.2, 11.5, 13.2, 14.9, 17.8, 21.0, 22.9, 25.2, 28.6, 31.3, 34.9, 40.7, 45.4, 48.8, 52.1, 55.9, 57.8, 60.3, 62.1, 65.2, 69.0, 72.1, 76.4, 78.7, 81.6, 83.6, 86.7, 88.6, 91.4, 93.8, 99.7, 103.2, 105.8, 108.3, 109.7, 111.3, 114.6, 124.0, 130.2, 132.1, 137.1, 141.1, 149.8, 158.1, 160.3, 163.5, 177.1, 199.2, 201.2, 224.5, 238.7, 242.9, 248.7, 255.8, 259.8, 287.2, 291.0, 297.6, 298.8]
Movement events: [  3.3   4.9   6.4   8.3  11.4  13.4  15.   17.7  21.1  25.4  28.6  31.5
  35.4  40.6  45.4  48.8  52.1  55.9  60.1  62.3  65.   69.1  72.4  76.3
  78.9  81.6  84.4  86.7  88.6  91.5  93.8  99.5 103.1 105.9 111.3 114.5
 118.4 123.9 130.2 137.1 139.1 140.8 144.  145.9 149.7 154.9 160.2 164.6
 168.8 170.7 173.1 176.9 181.2 187.5 190.1 195.4 201.2 207.5 210.5 216.8
 221.9 224.7 227.  231.6 235.  236.2 239.6 243.  248.6 250.8 257.5 261.6
 266.6 271.  273.  276.1 280.1 287.6 291.6 295.1 296.4 298. ]
Total real events:  62 Total predicted events:  82 Count: 46
Sensitivit

In [7]:
#Calculating predicted events to the nearest true movement event

import pandas as pd

min_dist = []
for i in range(len(event3_times)):
    temp=np.NaN
    for j in range(len(bin_events)):
        if(j==1):
            temp = abs(event3_times[i]-bin_events[j])
        else:
            if(abs(event3_times[i]-bin_events[j])<temp):
                temp=abs(event3_times[i]-bin_events[j])
            #endif
        #endif
    #endfor j
    min_dist.append(np.round(temp,1))
#endfor i

print("Min dist: ", min_dist)
df=pd.DataFrame(min_dist)
df.to_csv('20241220_min_dist_init_peaks.csv', header=False)

Min dist:  [1.7, 0.1, 0.1, 0.1, 0.1, 0.2, 0.1, 0.1, 0.1, 0.2, 0.0, 0.2, 0.5, 0.1, 0.0, 0.0, 0.0, 0.0, 0.2, 0.2, 0.2, 0.1, 0.3, 0.1, 0.2, 0.0, 0.8, 0.0, 0.0, 0.1, 0.0, 0.2, 0.1, 0.1, 0.0, 0.1, 3.8, 0.1, 0.0, 0.0, 2.0, 0.3, 2.9, 3.9, 0.1, 3.2, 0.1, 1.1, 5.3, 6.4, 4.0, 0.2, 4.1, 10.4, 9.1, 3.8, 0.0, 6.3, 9.3, 7.7, 2.6, 0.2, 2.5, 7.1, 3.7, 2.5, 0.9, 0.1, 0.1, 2.1, 1.7, 1.8, 6.8, 11.2, 13.2, 11.1, 7.1, 0.4, 0.6, 2.5, 1.2, 0.4]


In [11]:
#Calcuate different correlation coefficient cutoffs

diff = 0.5 #the cutoff time difference in s below which is considered the same event

peaks = np.load("20241220_init_peaks.npy")
corr_coeff = np.load("20241220_init_peaks_corr_coeff.npy")
time_data = np.load("20241220_init_time_data.npy")

#Cutoff for peak detection
corr_cutoff=[-1,-0.75,-0.5,-0.25,0,0.25,0.5,0.75,1]
for k in range(len(corr_cutoff)):
    pruned_peaks=[]
    for j in range(len(peaks)):
        if(corr_coeff[peaks[j]]>corr_cutoff[k]):
            pruned_peaks.append(peaks[j])
        #endif
    #endfor j

    event3_times=[]
    for j in range(len(pruned_peaks)):
        event3_times.append(time_data[pruned_peaks[j]])
    #endfor j
   
    bin_pos = 0
    bin_events = []
    for j in range(len(all_movement_bin)):
        if(all_movement_bin[j]>0):
            if(bin_pos==0):
                bin_pos=1
                bin_events.append(j/10)
        else:
            bin_pos=0
        #endif
    #endfor j
    #print("binary events:", bin_events)
    #print("movement events:", np.round(event3_times,1))

    #find nearest neighbor
    count=0
    diff = 0.5
    unique_times = []
    for i in range(len(event3_times)):
        for j in range(len(bin_events)):
            if(abs(event3_times[i]-bin_events[j])<diff):
                count=count+1
                unique_times.append(bin_events[j])
                break
            #endif
        #endfor j
    #endfor i
    unique_times = np.unique(unique_times)
    count=len(unique_times)
    count2=len(event3_times)-count
    print("\033[1m"+str(k+1)+". Correlation cutoff: "+str(corr_cutoff[k])+"\033[0m")
    print("Real events: ", len(bin_events), "Predicted events: ", len(event3_times), "Count:", count) 
    print("FPR: ", np.round(count2/(count2+300*frate-len(bin_events)),3), "TPR: ", np.round(count/len(bin_events),3))  
    print()
#endfor k

1. Correlation cutoff: -1
Real events:  62 Predicted events:  90 Count: 51
FPR:  0.013 TPR:  0.823

2. Correlation cutoff: -0.75
Real events:  62 Predicted events:  90 Count: 51
FPR:  0.013 TPR:  0.823

3. Correlation cutoff: -0.5
Real events:  62 Predicted events:  90 Count: 51
FPR:  0.013 TPR:  0.823

4. Correlation cutoff: -0.25
Real events:  62 Predicted events:  88 Count: 51
FPR:  0.012 TPR:  0.823

5. Correlation cutoff: 0
Real events:  62 Predicted events:  82 Count: 46
FPR:  0.012 TPR:  0.742

6. Correlation cutoff: 0.25
Real events:  62 Predicted events:  69 Count: 40
FPR:  0.01 TPR:  0.645

7. Correlation cutoff: 0.5
Real events:  62 Predicted events:  57 Count: 33
FPR:  0.008 TPR:  0.532

8. Correlation cutoff: 0.75
Real events:  62 Predicted events:  42 Count: 26
FPR:  0.005 TPR:  0.419

9. Correlation cutoff: 1
Real events:  62 Predicted events:  0 Count: 0
FPR:  0.0 TPR:  0.0



# TPR and TFR for movement offset predictions

In [16]:
#Finding true positive rate and true negative rate for the predictions
#select mouse
k=1
diff = 0.5 #the cutoff time difference in s below which is considered the same event

event3_times = np.load("20241220_arrest_peak_times.npy")
#Get video information
SELECT_VIDEO_PATH = VIDEO_PATH[k]
print("Collecting video information...")
cap = cv2.VideoCapture(SELECT_VIDEO_PATH)
fwidth = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
fheight = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
frate = cap.get(cv2.CAP_PROP_FPS)
tframes = cap.get(cv2.CAP_PROP_FRAME_COUNT)
cap.release()
cv2.destroyAllWindows()

#finding all movement events
bin_pos = 0
bin_events = []
for i in range(len(all_movement_bin)):
    if(all_movement_bin[i]>0):
            bin_pos=1
    else:
        if(bin_pos==1):
            bin_events.append(i/10)
        #endif
        bin_pos=0
    #endif
#endfor i
print("True movement events:", bin_events)
print("Predicted movement offset events:", np.round(event3_times,1))

#nearest neighbor
count=0
for i in range(len(event3_times)):
    for j in range(len(bin_events)):
        if(abs(event3_times[i]-bin_events[j])<diff):
            count=count+1
            break
        #endif
    #endfor j
#endfor i
print("Total real events: ", len(bin_events), "Total predicted events: ", len(event3_times), "Count:", count) 
print("Sensitivity or TPR: ", np.round(count/len(bin_events),3))
count2=len(event3_times)-count
print("False positive rate: ", np.round(count2/(count2+300*frate-len(bin_events)),3))


True movement events: [2.4, 5.5, 7.3, 8.7, 12.3, 14.1, 15.5, 18.4, 22.3, 23.9, 25.7, 29.2, 34.2, 36.1, 41.3, 46.7, 51.0, 52.8, 56.9, 58.3, 60.9, 62.9, 66.3, 70.9, 74.2, 77.3, 79.5, 82.4, 84.9, 87.4, 89.8, 91.9, 96.5, 101.7, 103.7, 107.5, 108.9, 110.2, 112.0, 116.3, 124.5, 130.9, 134.3, 139.7, 141.6, 157.1, 158.9, 160.9, 175.1, 197.4, 199.7, 223.8, 236.2, 240.7, 246.8, 254.6, 259.1, 284.1, 290.0, 296.0, 298.1, 299.4]
Predicted movement offset events: [  3.6   5.2   6.9   9.4  12.1  13.8  15.3  18.2  21.8  24.   26.3  27.4
  29.   32.1  33.9  36.1  38.6  41.5  43.6  46.7  49.5  50.9  52.5  56.5
  58.3  60.6  63.1  66.1  69.8  73.2  77.1  79.4  82.3  84.8  87.2  89.7
  91.9  95.9 100.8 103.6 107.  108.5 109.8 112.1 115.4 120.5 124.4 127.5
 130.9 133.6 137.8 139.5 141.2 144.6 146.5 150.8 152.6 155.3 158.6 160.5
 165.7 167.5 169.2 171.  173.6 175.7 177.8 181.5 183.7 188.  190.9 193.5
 196.2 201.9 203.4 209.2 212.3 215.1 217.9 220.5 223.1 225.3 227.8 232.4
 235.1 236.5 240.3 244.3 246.5 248.

In [17]:
#Calculating predicted events to the nearest true movement event

import pandas as pd

min_dist = []
for i in range(len(event3_times)):
    temp=np.NaN
    for j in range(len(bin_events)):
        if(j==1):
            temp = abs(event3_times[i]-bin_events[j])
        else:
            if(abs(event3_times[i]-bin_events[j])<temp):
                temp=abs(event3_times[i]-bin_events[j])
            #endif
        #endif
    #endfor j
    min_dist.append(np.round(temp,1))
#endfor i

print("Min dist: ", min_dist)
df=pd.DataFrame(min_dist)
df.to_csv('min_dist_arrest.csv', header=False)

Min dist:  [1.9, 0.3, 0.4, 0.7, 0.2, 0.3, 0.2, 0.2, 0.5, 0.1, 0.6, 1.7, 0.2, 2.1, 0.3, 0.0, 2.5, 0.2, 2.3, 0.0, 1.5, 0.1, 0.3, 0.4, 0.0, 0.3, 0.2, 0.2, 1.1, 1.0, 0.2, 0.1, 0.1, 0.1, 0.2, 0.1, 0.0, 0.6, 0.9, 0.1, 0.5, 0.4, 0.4, 0.1, 0.9, 4.0, 0.1, 3.0, 0.0, 0.7, 1.9, 0.2, 0.4, 3.0, 4.9, 6.3, 4.5, 1.8, 0.3, 0.4, 4.8, 6.6, 5.9, 4.1, 1.5, 0.6, 2.7, 6.4, 8.6, 9.4, 6.5, 3.9, 1.2, 2.2, 3.7, 9.5, 11.5, 8.7, 5.9, 3.3, 0.7, 1.5, 4.0, 3.8, 1.1, 0.3, 0.4, 2.5, 0.3, 2.1, 0.7, 1.0, 3.1, 4.4, 5.6, 8.0, 12.3, 9.4, 7.1, 3.6, 1.0, 0.7, 1.0, 2.7, 0.9, 0.5]


In [19]:
#Calcuate different correlation coefficient cutoffs

diff = 0.5 #the cutoff time difference in s below which is considered the same event

peaks = np.load("20241220_arrest_peaks.npy")
corr_coeff = np.load("20241220_arrest_peaks_corr_coeff.npy")
time_data = np.load("20241220_arrest_time_data.npy")

#Cutoff for peak detection
corr_cutoff=[-1,-0.75,-0.5,-0.25,0,0.25,0.5,0.75,1]
for k in range(len(corr_cutoff)):
    pruned_peaks=[]
    for j in range(len(peaks)):
        if(corr_coeff[peaks[j]]>corr_cutoff[k]):
            pruned_peaks.append(peaks[j])
        #endif
    #endfor j

    event3_times=[]
    for j in range(len(pruned_peaks)):
        event3_times.append(time_data[pruned_peaks[j]])
    #endfor j
   
    bin_pos = 0
    bin_events = []
    for i in range(len(all_movement_bin)):
        if(all_movement_bin[i]>0):
            bin_pos=1
        else:
            if(bin_pos==1):
                bin_events.append(i/10)
            #endif
            bin_pos=0
        #endif
    #endfor i
    #endfor j
    #print("binary events:", bin_events)
    #print("movement events:", np.round(event3_times,1))
    
    #find nearest neighbor
    count=0
    diff = 0.5
    unique_times = []
    for i in range(len(event3_times)):
        for j in range(len(bin_events)):
            if(abs(event3_times[i]-bin_events[j])<diff):
                count=count+1
                unique_times.append(bin_events[j])
                break
            #endif
        #endfor j
    #endfor i
    unique_times = np.unique(unique_times)
    count=len(unique_times)
    count2=len(event3_times)-count
    print("\033[1m"+str(k+1)+". Correlation cutoff: "+str(corr_cutoff[k])+"\033[0m")
    print("Real events: ", len(bin_events), "Predicted events: ", len(event3_times), "Count:", count) 
    print("FPR: ", np.round(count2/(count2+300*frate-len(bin_events)),3), "TPR: ", np.round(count/len(bin_events),3))  
    print()
#endfor k

1. Correlation cutoff: -1
Real events:  62 Predicted events:  109 Count: 40
FPR:  0.023 TPR:  0.645

2. Correlation cutoff: -0.75
Real events:  62 Predicted events:  109 Count: 40
FPR:  0.023 TPR:  0.645

3. Correlation cutoff: -0.5
Real events:  62 Predicted events:  109 Count: 40
FPR:  0.023 TPR:  0.645

4. Correlation cutoff: -0.25
Real events:  62 Predicted events:  109 Count: 40
FPR:  0.023 TPR:  0.645

5. Correlation cutoff: 0
Real events:  62 Predicted events:  106 Count: 40
FPR:  0.022 TPR:  0.645

6. Correlation cutoff: 0.25
Real events:  62 Predicted events:  90 Count: 37
FPR:  0.018 TPR:  0.597

7. Correlation cutoff: 0.5
Real events:  62 Predicted events:  63 Count: 26
FPR:  0.012 TPR:  0.419

8. Correlation cutoff: 0.75
Real events:  62 Predicted events:  17 Count: 6
FPR:  0.004 TPR:  0.097

9. Correlation cutoff: 1
Real events:  62 Predicted events:  0 Count: 0
FPR:  0.0 TPR:  0.0

